In [2]:
import os 
import random
import shutil
from shutil import copyfile

In [3]:
import numpy as np
import pandas as pd 
import random
import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [7]:
import tensorflow as tf 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from tensorflow.keras import Model
import matplotlib.pyplot as plt 
from matplotlib.offsetbox import (TextArea,DrawingArea,OffsetImage,AnnotationBbox)
import matplotlib.patches as mpatches
from sklearn.utils import shuffle
from tqdm import tqdm 

In [8]:
base_dir = 'C:/Users/SEISANG-G06/Documents/LINN/DATA_CLEANING/BT'
os.chdir(base_dir)

In [12]:
df = pd.read_csv("Brain Tumor.csv")[["Image","Class","Mean"]]
display(df.head())
print(df.shape)

,Image,Class,Mean
0,Image1,0,6.535339
1,Image2,0,8.749969
2,Image3,1,7.341095
3,Image4,1,5.958145
4,Image5,0,7.315231


(3762, 3)


# Data Splitting

In [35]:
df_full_train,df_test = train_test_split(df,test_size=0.2,random_state=11)
df_train,df_val = train_test_split(df_full_train,test_size=0.25,random_state=11)

In [71]:
df_test.shape

(753, 3)

In [72]:
df_val.shape

(753, 3)

In [36]:
train_labels = df_train['Class'].values
train_file_names = df_train['Image'].values

In [37]:
val_labels = df_val['Class'].values
val_file_names = df_val['Image'].values

In [38]:
test_labels = df_test['Class'].values
test_file_names = df_test['Image'].values

In [46]:
def split_array_labels(arr_image,arr_label):
    arr_image_0 = arr_image[np.where(arr_label==0)]
    arr_image_1 = arr_image[np.where(arr_label==1)]
    return {'0':arr_image_0,'1':arr_image_1}

In [51]:
train_arr_dict = split_array_labels(train_file_names,train_labels)
val_arr_dict = split_array_labels(val_file_names,val_labels)
test_arr_dict = split_array_labels(test_file_names,test_labels)

In [74]:
def create_empty_space(base_dir):
    check_exist_path = os.path.join(base_dir,'_MODELLING')
    
    if os.path.isdir(check_exist_path):
        shutil.rmtree(check_exist_path)
        print("Remove old directories")
        
    for label in ['0','1']:
        train_dir = os.path.join(base_dir,'_MODELLING','training',label)
        val_dir = os.path.join(base_dir,'_MODELLING','validation',label)
        test_dir = os.path.join(base_dir,'_MODELLING','testing',label)
        
        
        os.makedir(train_dir)
        os.makedir(val_dir)
        os.makedir(test_dir)
    print(f"Create empty training,validation and testing directories")
create_empty_directories(base_dir)

Remove old directories
Created empty training, validation and testing directories


In [75]:
def split_data(source_dir,train_arr_dict,val_arr_dict,test_arr_dict):
    for label in tqdm(['0','1']):
        for filename in train_arr_dict[label]:
            file_name=f"{filename}.jpg"
            source = os.path.join(source_dir,'Brain Tumor','Brain Tumor',file_name)
            destination = os.path.join(base_dir,'_MODELLING','training',label,file_name)
            copyfile(source,destination)
            
        for filename in val_arr_dict[label]:
            file_name = f"{filename}.jpg"
            source = os.path.join(source_dir,'Brain Tumor','Brain Tumor',file_name)
            destination = os.path.join(base_dir,'_MODELLING','validation',label,file_name)
            copyfile(source,destination)
            
        for filename in test_arr_dict[label]:
            file_name = f"{filename}.jpg"
            source = os.path.join(source_dir,'Brain Tumor','Brain Tumor',file_name)
            destination = os.path.join(base_dir,'_MODELLING','testing',label,file_name)
            copyfile(source,destination)
        print(f"Created training,validation and testing directories containing images")

split_data(base_dir,train_arr_dict,val_arr_dict,test_arr_dict)
            

 50%|█████     | 1/2 [00:01<00:01,  1.20s/it]

Created training,validation and testing directories containing images


100%|██████████| 2/2 [00:02<00:00,  1.16s/it]

Created training,validation and testing directories containing images


In [182]:
model_dir = 'C:/Users/SEISANG-G06/Documents/LINN/DATA_CLEANING/BT/_MODELLING'
os.chdir(model_dir)

mytrain_test_val_list= []
mylabel_list = []
myitem_list = []
vis_images = []
vis_labels= [ ]

train_test_val = tf.io.gfile.listdir(model_dir)
for folder in train_test_val:
    path1 = os.path.join(model_dir,folder)
    label_list = tf.io.gfile.listdir(path1)
    for label in label_list:
        file_path = os.path.join(path1,label)
        item = os.listdir(file_path)
        
        
        mytrain_test_val_list.append(folder)
        mylabel_list.append(label)
        myitem_list.append(len(item))
        
        my_files = item[0:5]
        for file in my_files:
            vis_image = os.path.join(file_path,file)
            vis_labels.append(label)
            vis_images.append(vis_image)
pd.DataFrame({"Train Test":mytrain_test_val_list,"Labels":mylabel_list,'Number of Items':myitem_list})

,Train Test,Labels,Number of Items
0,testing,0,404
1,testing,1,349
2,training,0,1257
3,training,1,999
4,validation,0,418
5,validation,1,335


# Data Augmentation

In [214]:
def train_val_generators(TRAINING_DIR,VALDATION_DIR,TEST_DIR):
    train_datagen = ImageDataGenerator(rescale=1./127.5,
                                       rotation_range=30,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,batch_size=32,class_mode='binary',target_size=(150,150))
    
    valid_or_test_datagen = ImageDataGenerator(rescale=1./127.5)
    validation_generator = valid_or_test_datagen.flow_from_directory(directory=VALDATION_DIR,batch_size=32,class_mode='binary',
                                                                     target_size=(150,150))
    
    test_generator = valid_or_test_datagen.flow_from_directory(directory=TEST_DIR,batch_size=32,class_mode='binary',
                                                               target_size=(150,150))
    
    return train_generator,validation_generator,test_generator

In [215]:
training_dir = os.path.join(base_dir,'training')
validation_dir = os.path.join(base_dir,'validation')
testing_dir = os.path.join(base_dir,'testing')

print(testing_dir)

C:/Users/SEISANG-G06/Documents/LINN/DATA_CLEANING/BT/_MODELLING\testing


In [216]:
train_generator,validation_generator,test_generator = train_val_generators(training_dir,validation_dir,testing_dir)

Found 2256 images belonging to 2 classes.
Found 753 images belonging to 2 classes.
Found 753 images belonging to 2 classes.


In [218]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(150,150,3),include_top=False,weights='imagenet')

base_model.trainable = False
last_output = base_model.output
num_trainable_params = sum([w.shape.num_elements()for w in base_model.trainable_weights])

print(f"There are {num_trainable_params:,} trainable parameters in this model.")
print(f"The pretrained model has type: {type(base_model)}")



9406464/9406464 [==============================] - 2s 0us/step
There are 0 trainable parameters in this model.
The pretrained model has type: <class 'keras.src.engine.functional.Functional'>


# Building Deep Neural Network Architecture with MobileNetV2

In [219]:
def transfer_learning(last_output,pre_trained_model):
    x=tf.keras.layers.Flatten()(last_output)
    x=tf.keras.layers.Dense(1024,activation='relu')(x)
    x=tf.keras.layers.Dropout(0.6)(x)
    x=tf.keras.layers.Dense(1,activation='sigmoid')(x)
    model = Model(inputs=pre_trained_model.input,outputs=x)
    
    return model

model = transfer_learning(last_output,base_model)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 75, 75, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 75, 75, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 75, 75, 32)           0         ['bn_Conv1[0][0]']        

In [220]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003),loss='binary_crossentropy',metrics=['accuracy'])

In [222]:
history = model.fit(train_generator,epochs=10,validation_data=validation_generator,
                    callbacks=[callback])

Epoch 1/10


71/71 [==============================] - 63s 854ms/step - loss: 1.8075 - accuracy: 0.8001 - val_loss: 0.4023 - val_accuracy: 0.8672
Epoch 2/10
71/71 [==============================] - 55s 777ms/step - loss: 0.3789 - accuracy: 0.8564 - val_loss: 0.2822 - val_accuracy: 0.8792
Epoch 3/10
71/71 [==============================] - 55s 776ms/step - loss: 0.3141 - accuracy: 0.8741 - val_loss: 0.2930 - val_accuracy: 0.8818
Epoch 4/10
71/71 [==============================] - 56s 783ms/step - loss: 0.3045 - accuracy: 0.8865 - val_loss: 0.2743 - val_accuracy: 0.8818
Epoch 5/10
71/71 [==============================] - 55s 779ms/step - loss: 0.3109 - accuracy: 0.8759 - val_loss: 0.2703 - val_accuracy: 0.8871
Epoch 6/10
71/71 [==============================] - 56s 781ms/step - loss: 0.2940 - accuracy: 0.8790 - val_loss: 0.2569 - val_accuracy: 0.8765
Epoch 7/10
71/71 [==============================] - 55s 777ms/step - loss: 0.2944 - accuracy: 0.8821 - val_loss: 0.2946 - val_accuracy: 0.88